In [2]:
import os
import pandas as pd
import shutil

In [3]:
image_folders = ['NIH_DATASET/images_001', 'NIH_DATASET/images_002','NIH_DATASET/images_003','NIH_DATASET/images_004','NIH_DATASET/images_005','NIH_DATASET/images_006','NIH_DATASET/images_007','NIH_DATASET/images_008','NIH_DATASET/images_009','NIH_DATASET/images_010','NIH_DATASET/images_011','NIH_DATASET/images_012']  
annotations_csv_path = 'NIH_DATASET/BBox_List_2017.csv'
output_folder = 'NIH_DATASET/filtered_images'
filtered_csv_path = 'NIH_DATASET/filtered_csv_path.csv' 


In [4]:
annotations_df = pd.read_csv(annotations_csv_path)
annotations_df = pd.read_csv(annotations_csv_path)
print(annotations_df.columns)


Index(['Image Index', 'Finding Label', 'Bbox [x', 'y', 'w', 'h]', 'Unnamed: 6',
       'Unnamed: 7', 'Unnamed: 8'],
      dtype='object')


In [5]:
filtered_annotations = []
for index, row in annotations_df.iterrows():
    image_name = row['Image Index']
    found_image = False
    
    for folder in image_folders:
        image_path = os.path.join(folder, 'images', image_name)  # Adjusted to include 'images' subfolder
        print(f"Checking for image: {image_path}")
        
        if os.path.exists(image_path):
            print(f"Found and copying image: {image_path}")
            shutil.copy(image_path, output_folder)
            filtered_annotations.append(row)
            found_image = True
            break

    if not found_image:
        print(f"Image not found: {image_name}")


Checking for image: NIH_DATASET/images_001/images/00013118_008.png
Checking for image: NIH_DATASET/images_002/images/00013118_008.png
Checking for image: NIH_DATASET/images_003/images/00013118_008.png
Checking for image: NIH_DATASET/images_004/images/00013118_008.png
Checking for image: NIH_DATASET/images_005/images/00013118_008.png
Checking for image: NIH_DATASET/images_006/images/00013118_008.png
Found and copying image: NIH_DATASET/images_006/images/00013118_008.png
Checking for image: NIH_DATASET/images_001/images/00014716_007.png
Checking for image: NIH_DATASET/images_002/images/00014716_007.png
Checking for image: NIH_DATASET/images_003/images/00014716_007.png
Checking for image: NIH_DATASET/images_004/images/00014716_007.png
Checking for image: NIH_DATASET/images_005/images/00014716_007.png
Checking for image: NIH_DATASET/images_006/images/00014716_007.png
Checking for image: NIH_DATASET/images_007/images/00014716_007.png
Found and copying image: NIH_DATASET/images_007/images/00

In [6]:
filtered_annotations

[Image Index      00013118_008.png
 Finding Label         Atelectasis
 Bbox [x                225.084746
 y                      547.019217
 w                       86.779661
 h]                      79.186441
 Unnamed: 6                    NaN
 Unnamed: 7                    NaN
 Unnamed: 8                    NaN
 Name: 0, dtype: object,
 Image Index      00014716_007.png
 Finding Label         Atelectasis
 Bbox [x                686.101695
 y                      131.543498
 w                      185.491525
 h]                     313.491525
 Unnamed: 6                    NaN
 Unnamed: 7                    NaN
 Unnamed: 8                    NaN
 Name: 1, dtype: object,
 Image Index      00029817_009.png
 Finding Label         Atelectasis
 Bbox [x                221.830508
 y                      317.053115
 w                      155.118644
 h]                     216.949153
 Unnamed: 6                    NaN
 Unnamed: 7                    NaN
 Unnamed: 8                    NaN
 Name

In [7]:
print(f"Total filtered annotations: {len(filtered_annotations)}")


Total filtered annotations: 984


In [8]:
filtered_annotations_df = pd.DataFrame(filtered_annotations)
filtered_annotations_df.to_csv(filtered_csv_path, index=False)
print(f"Filtered annotations saved to: {filtered_csv_path}")

Filtered annotations saved to: NIH_DATASET/filtered_csv_path.csv


In [9]:
data_entrylist_df = pd.read_csv('NIH_DATASET/Data_Entry_2017.csv')
all_labels = data_entrylist_df['Finding Labels'].str.split('|').explode().unique()
print(all_labels)


['Atelectasis' 'Cardiomegaly' 'Consolidation' 'Edema' 'Effusion'
 'Infiltration' 'Mass' 'Nodule' 'Pleural_Thickening' 'Pneumonia'
 'Pneumothorax' 'Fibrosis' 'Emphysema' 'Hernia' 'No Finding']


In [10]:
import os
import json
from PIL import Image
import pandas as pd
from detectron2.data.datasets import register_coco_instances


In [11]:
categories = [
    {"id": 1, "name": "Atelectasis"},
    {"id": 2, "name": "Cardiomegaly"},
    {"id": 3, "name": "Consolidation"},
    {"id": 4, "name": "Edema"},
    {"id": 5, "name": "Effusion"},
    {"id": 6, "name": "Emphysema"},
    {"id": 7, "name": "Fibrosis"},
    {"id": 8, "name": "Hernia"},
    {"id": 9, "name": "Infiltration"},
    {"id": 10, "name": "Mass"},
    {"id": 11, "name": "Nodule"},
    {"id": 12, "name": "Pleural_Thickening"},
    {"id": 13, "name": "Pneumonia"},
    {"id": 14, "name": "Pneumothorax"},
    {"id": 15, "name": "No Finding"}
]


In [12]:
def convert_to_coco(annotations_df, output_json_path, image_dir, categories):
    images = []
    annotations = []
    category_name_to_id = {category['name']: category['id'] for category in categories}

    image_id = 0
    annotation_id = 0

    for index, row in annotations_df.iterrows():
        image_name = row['Image Index']
        image_path = os.path.join(image_dir, image_name)

        with Image.open(image_path) as img:
            width, height = img.size

        images.append({
            'file_name': image_name,
            'height': height,
            'width': width,
            'id': image_id
        })

        findings = row['Finding Label'].split('|')
        for finding in findings:
            if finding not in category_name_to_id:
                continue  
            category_id = category_name_to_id[finding]
            
            # Extract bounding box coordinates
            bbox_x = float(row['Bbox [x'])
            bbox_y = float(row['y'])
            bbox_w = float(row['w'])
            bbox_h = float(row['h]'])

            bbox = [bbox_x, bbox_y, bbox_w, bbox_h]

            annotations.append({
                'id': annotation_id,
                'image_id': image_id,
                'category_id': category_id,
                'bbox': bbox,
                'area': bbox_w * bbox_h,
                'iscrowd': 0
            })
            annotation_id += 1

        image_id += 1

    coco_format = {
        'images': images,
        'annotations': annotations,
        'categories': categories
    }

    with open(output_json_path, 'w') as f:
        json.dump(coco_format, f, indent=4)

In [13]:
filtered_annotations_df = pd.read_csv('NIH_DATASET/filtered_csv_path.csv')
output_folder = 'NIH_DATASET/filtered_images'


In [14]:
convert_to_coco(filtered_annotations_df, 'NIH_DATASET/output_json_path.json', output_folder, categories)


In [15]:
import os
import json
from detectron2.structures import BoxMode
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data.datasets import register_coco_instances

In [16]:
def register_dataset(dataset_name, json_file, image_root):
    register_coco_instances(dataset_name, {}, json_file, image_root)


In [17]:
register_dataset("nih_dataset_detectron2", "NIH_DATASET/output_json_path.json", "NIH_DATASET/filtered_images")


In [18]:
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer
from detectron2 import model_zoo

In [19]:
cfg = get_cfg()
cfg.DATASETS.TRAIN = ("nih_dataset_detectron2",)
cfg.DATASETS.TEST = ()

In [20]:
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer
from detectron2 import model_zoo
from detectron2.data.datasets import register_coco_instances

In [21]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))


In [22]:
cfg.DATASETS.TRAIN = ("nih_dataset_detectron2",)
cfg.DATASETS.TEST = ()  
cfg.DATALOADER.NUM_WORKERS = 2  
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 14  
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 1000
cfg.OUTPUT_DIR = "./output"

In [23]:
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[07/09 22:11:07 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (15, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (15,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (56, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (56,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias

[07/09 22:11:08 d2.engine.train_loop]: Starting training from iteration 0


/data/anaconda/envs/test/lib/python3.8/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[07/09 22:11:14 d2.utils.events]:  eta: 0:04:49  iter: 19  total_loss: 3.01  loss_cls: 2.842  loss_box_reg: 0.02517  loss_rpn_cls: 0.1148  loss_rpn_loc: 0.01029    time: 0.2911  last_time: 0.2642  data_time: 0.0106  last_data_time: 0.0027   lr: 4.9953e-06  max_mem: 1732M
[07/09 22:11:20 d2.utils.events]:  eta: 0:04:42  iter: 39  total_loss: 2.72  loss_cls: 2.581  loss_box_reg: 0.03866  loss_rpn_cls: 0.07631  loss_rpn_loc: 0.008775    time: 0.2920  last_time: 0.3228  data_time: 0.0028  last_data_time: 0.0027   lr: 9.9902e-06  max_mem: 1733M
[07/09 22:11:26 d2.utils.events]:  eta: 0:04:36  iter: 59  total_loss: 2.192  loss_cls: 2.066  loss_box_reg: 0.02734  loss_rpn_cls: 0.07117  loss_rpn_loc: 0.008502    time: 0.2923  last_time: 0.2759  data_time: 0.0027  last_data_time: 0.0027   lr: 1.4985e-05  max_mem: 1733M
[07/09 22:11:32 d2.utils.events]:  eta: 0:04:31  iter: 79  total_loss: 1.551  loss_cls: 1.326  loss_box_reg: 0.04223  loss_rpn_cls: 0.1084  loss_rpn_loc: 0.01038    time: 0.2939  

In [24]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
import detectron2.utils.comm as comm

In [25]:
evaluator = COCOEvaluator("nih_dataset_detectron2", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "nih_dataset_detectron2")


WARNING [07/09 22:16:18 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.
[07/09 22:16:18 d2.data.datasets.coco]: Loaded 984 images in COCO format from NIH_DATASET/output_json_path.json
[07/09 22:16:18 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[07/09 22:16:18 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[07/09 22:16:18 d2.data.common]: Serializing 984 elements to byte tensors and concatenating them all ...
[07/09 22:16:18 d2.data.common]: Serialized dataset takes 0.23 MiB


In [26]:
inference_on_dataset(trainer.model, val_loader, evaluator)

[07/09 22:16:18 d2.evaluation.evaluator]: Start inference on 984 batches
[07/09 22:16:19 d2.evaluation.evaluator]: Inference done 11/984. Dataloading: 0.0009 s/iter. Inference: 0.0789 s/iter. Eval: 0.0001 s/iter. Total: 0.0799 s/iter. ETA=0:01:17
[07/09 22:16:24 d2.evaluation.evaluator]: Inference done 73/984. Dataloading: 0.0010 s/iter. Inference: 0.0797 s/iter. Eval: 0.0001 s/iter. Total: 0.0808 s/iter. ETA=0:01:13
[07/09 22:16:29 d2.evaluation.evaluator]: Inference done 135/984. Dataloading: 0.0010 s/iter. Inference: 0.0798 s/iter. Eval: 0.0001 s/iter. Total: 0.0810 s/iter. ETA=0:01:08
[07/09 22:16:34 d2.evaluation.evaluator]: Inference done 197/984. Dataloading: 0.0010 s/iter. Inference: 0.0798 s/iter. Eval: 0.0002 s/iter. Total: 0.0810 s/iter. ETA=0:01:03
[07/09 22:16:39 d2.evaluation.evaluator]: Inference done 259/984. Dataloading: 0.0010 s/iter. Inference: 0.0797 s/iter. Eval: 0.0002 s/iter. Total: 0.0809 s/iter. ETA=0:00:58
[07/09 22:16:44 d2.evaluation.evaluator]: Inference do

OrderedDict([('bbox',
              {'AP': 0.9286000215212994,
               'AP50': 3.196567658506665,
               'AP75': 0.2253085697278584,
               'APs': 0.0,
               'APm': 0.0,
               'APl': 0.92868959606467,
               'AP-Atelectasis': 0.0,
               'AP-Cardiomegaly': 6.500200150649098,
               'AP-Consolidation': nan,
               'AP-Edema': nan,
               'AP-Effusion': 0.0,
               'AP-Emphysema': nan,
               'AP-Fibrosis': nan,
               'AP-Hernia': nan,
               'AP-Infiltration': nan,
               'AP-Mass': 0.0,
               'AP-Nodule': 0.0,
               'AP-Pleural_Thickening': nan,
               'AP-Pneumonia': 0.0,
               'AP-Pneumothorax': 0.0,
               'AP-No Finding': nan})])